
# CT421 Project 1  Evolutionary Search - GAs

**Aoife Mulligan 20307646 | Leo Chui 20343266**  


### 1.1 OneMax Problem

In [67]:
from random import random, randrange


# Define global variables for string size, population size, and mutation rate

STRING_SIZE = 30
POPULATION_SIZE = 100
MUTATION_RATE = 1/STRING_SIZE
CROSSOVER_RATE = 0.9
NUM_GENERATIONS = 1000


In [68]:
'''# Generate random binary string of length 30

def generate_binary_string():
    return ''.join(rand.choice('01') for _ in range(30))

population = [generate_binary_string() for _ in range(10)]
print(population)
'''

'''for i in range(10):
    print(generate_binary_string())
    i += 1'''
    



'for i in range(10):\n    print(generate_binary_string())\n    i += 1'

In [69]:
# Tournament selection function

def tournament_selection(population, scores, k=3):
    # Select k individuals from population at random
    selection_i = randrange(len(population))
    for i in [randrange(len(population)) for _ in range(k-1)]:
        # Check if better (e.g. perform a tournament)
        if scores[i] > scores[selection_i]:
            selection_i = i
    # Return the index of the best
    return population[selection_i]

In [75]:
# Crossover function

def crossover(parent1, parent2, CROSSOVER_RATE):
    
    # Create default child
    child1, child2 = parent1[:], parent2[:]

    #check for recombination
    if random() < CROSSOVER_RATE:
        # selecr crossover point that is not the end of the string
        crossover_point = randrange(1, len(parent1) - 1)

        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return [child1, child2]
    

In [81]:
'''# Perform mutation

def mutation(bitstring, MUTATION_RATE):
    for i in range(len(bitstring)):
        
        # check for mutation
        if random() < MUTATION_RATE:
            bitstring[i] = 1 - int(bitstring[i])'''

def mutation(bitstring, MUTATION_RATE):
    bitlist = list(bitstring)  # Convert the binary string to a list
    for i in range(len(bitlist)):
        # Check for mutation
        if random() < MUTATION_RATE:
            # Convert the binary digit to an integer and perform mutation
            bitlist[i] = str(1 - int(bitlist[i]))
    
    # Convert the list back to a string (modify in place)
    bitstring = ''.join(bitlist)

In [78]:
# Calculate fitness

def fitness(bitstring):
    return -sum(int(bit) for bit in bitstring)


In [82]:
def algorithm(fitness, STRING_SIZE, NUM_GENERATIONS, POPULATION_SIZE, CROSSOVER_RATE, MUTATION_RATE):
    # Initial random population
    population = [''.join([str(randrange(2)) for _ in range(STRING_SIZE)]) for _ in range(POPULATION_SIZE)]
    # Store best solution
    best_solution, best_fitness = 0, fitness(population[0])
    # Enumerate generations
    for gen in range(NUM_GENERATIONS):
        # Evaluate all solutions in the population
        scores = [fitness(c) for c in population]
        # Check for new best solution
        for i in range(POPULATION_SIZE):
            if scores[i] < best_fitness:
                best_solution, best_fitness = population[i], scores[i]
                print(">%d, new best f(%s) = %.3f" % (gen,  population[i], scores[i]))
        # Select parents
        parents = [tournament_selection(population, scores) for i in range(POPULATION_SIZE)]

        # Create next generation
        children = list()
        for i in range(0, POPULATION_SIZE, 2):
            # Get pair of parents
            parent1, parent2 = parents[i], parents[i+1]
            # Perform crossover
            for child in crossover(parent1, parent2, CROSSOVER_RATE):
                # Perform mutation
                mutation(child, MUTATION_RATE)
                # Add to next generation
                children.append(child)
        # Replace population
        population = children
    return [best_solution, best_fitness]

In [83]:
# Perform the search

solution, score = algorithm(fitness, STRING_SIZE, NUM_GENERATIONS, POPULATION_SIZE, CROSSOVER_RATE, MUTATION_RATE)
print('Best Solution: f(%s) = %.5f' % (solution, score))

>0, new best f(110000111101110011010011110110) = -18.000
>0, new best f(010100010111111101110011011110) = -19.000
>0, new best f(101011010111001101111101101011) = -20.000
Best Solution: f(101011010111001101111101101011) = -20.00000
